In [1]:
import numpy as np
import math

t = 0  # number of this round
T = 800  # numbers of rounds
l = 30  # ||a|| <= L
dimension_arm = 3 #dimension of arm
dimension_super_arm = 9 #dimension of super arm
m = 1 #||theta|| < m

theta_stimato = [0.0, 0.0, 0.0] #np.array([])  theta
theta_stimato_super = [0.0] * dimension_super_arm

A = [] #collector of arm
num_arms = 0 #number of arms

Th = [] #collector of theta
num_theta = 0 #num theta

W = [] #distribution of probability to play arms
N = [] #number of times an arm is played
R = [] #collection of reward
H = [] #collection of hedge losses
L = [] #collection of losses
Mixability_Gap = [] #collection of all mixability gap at each t
AI = [] #number of the arm played at time I

nu = 0.01 #regularization paramether
eta = 0.1 #learning paramether
theta_vero = 0
curr_answ = 0

double_arm = [] #collector of double arm
num_double_arm = 0 #number of double arm
played_superarm = [] #number of times each superarm is played

Y = []
num_arms_y = 0

M = [] #world
num_M = 0

class world():
    matrix = np.array([])
    id

    def __init__(self, param):
        self.matrix = param
        global M, num_M
        self.id = num_M
        M.append(self)
        num_M +=1

def create_union_arm(id_x, id_y):
    res = []
    for i in range(0, K):
        for j in range(0, K):
            res.append(A[id_x].arm_vector[i] * Y[id_y].arm_vector[j])
    return res

def create_all_union():
    for i in range(0, num_arms):
        for j in range (0, num_arms_y):
            curr = create_union_arm(i, j)
            double_arm.append(curr)
            global num_double_arm
            num_double_arm += 1
            played_superarm.append(0)

def normalize(v):
    sum = np.sum(v)
    if sum > 0:
        return v / sum
    else:
        return np.array([0.33]*len(v))
    
class theta():
    vector = np.array([])
    best_arm = -1
    id

    def __init__(self, param):
        self.vector = param
        global Th, num_theta
        self.id = num_theta
        Th.append(self)
        num_theta +=1
        #i = 0
        #best = -5
        #while i < num_arms:
            #tmp = np.dot( A[i].arm_vector, np.transpose(self.vector))
            #if best < tmp:
                #best = tmp
                #self.best_arm = i

class arm():
    arm_vector = np.array([])
    id = 0
    best_theta = -1

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms, A
        A.append(self)
        W.append(0)
        N.append(0)
        L.append(0)
        self.id = num_arms
        num_arms +=1
        max = -5
        for i in range (0, num_theta):
            tmp = np.matmul(self.arm_vector, Th[i].vector)
            if tmp > max:
                self.best_theta = i
                max = tmp
                            
    def generate_double(self):
        for i in range(0, num_arms_y):
            double_arm.append(create_union_arm(self.id, i))    
                    
    def pull_arm(self):
        r = np.dot(self.arm_vector, np.transpose(Th[theta_vero].vector)) + np.random.normal(0, 1)
        R.append(r)
        N[self.id] += 1
        AI.append(self.id)
        return r
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)

class arm_y():
    arm_vector = np.array([])
    id = 0

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms_y, arm_Y
        Y.append(self)
        self.id = num_arms_y
        num_arms_y += 1    
    
def design_matrix(w):
    V = []
    i = 0
    while i < num_arms:
        b = [A[i].arm_vector]
        a = np.swapaxes(b, 0, 1)
        ris = np.matmul(a, b) * w[i]
        if (i == 0):
            V = ris
        else:
            V += ris
        i+=1
    return V

def norm_2_mat(x, V):
    b = [x]
    a = np.swapaxes(b, 0, 1)
    ris = np.sqrt(np.matmul(np.matmul(b,V),a))
    return ris[0][0]

def arm_to_play():
    diff = np.array(N) - np.array(W)
    i = 0
    arm = 0
    min = +5000
    while i < num_arms:
        if diff[i] < min:
            min = diff[i]
            arm = i
        i += 1
    return arm

def stima_theta():
    V = design_matrix(N)
    I = np.identity(dimension_arm)
    p = np.linalg.inv(V + nu*I)
    sum = 0
    for i in range (0, t):
        sum += R[i] * np.array(A[AI[i]].arm_vector)
    res = np.matmul(p, sum)
    global theta_stimato
    theta_stimato = res
    return res

def best_answer():
    max = -100
    best = -1
    for i in range (0, num_arms):
        curr = np.matmul(A[i].arm_vector, theta_stimato)
        if curr > max:
            max = curr
            best = i
    global curr_answ
    curr_answ = best
    return best

def best_arm():
    max = -100
    best = -1
    for i in range (0, num_arms):
        curr = np.matmul(A[i].arm_vector, Th[theta_vero].vector)
        if curr > max:
            max = curr
            best = i
    global curr_answ
    curr_answ = best
    return best


def beta(time, delta):
    n = np.abs(np.random.normal(0, 1))
    res = math.sqrt(math.log10(1/delta) + dimension_arm/2*math.log10(1 + (time*(l**2))/(n*dimension_arm))) + math.sqrt(n/2)*m
    return res

def best_response():
    min = 1000000
    theta_adv = -11
    for i in range (0, num_theta):
        if not i == A[curr_answ].best_theta:
            curr = (norm_2_mat(np.array(theta_stimato) - np.array(Th[i].vector), design_matrix(W)))**2
            if curr < min:
                min = curr
                theta_adv = i
    return theta_adv

def calcola_U (id_arm):
    xi = [0]*dimension_arm
    q2 = 4*(l**2)*(m**2)
    q1 = 0
    p1 = np.matmul((np.array(theta_stimato) - np.array(Th[best_response()].vector)), np.array(A[id_arm].arm_vector))
    p2 = math.sqrt(2*beta(T, 1/T**3))*norm_2_mat(np.array(A[id_arm].arm_vector), design_matrix(normalize(N)) + np.random.normal(0, 1)*np.identity(dimension_arm))
    r1 = (p1 + p2)**2
    r2 = (p1 - p2)**2
    if r1 > r2:
        q1 = r1
    else:
        q1 = r2
    if q1 > q2:
        return q2
    else:
        return q1

def calcola_loss():
    loss = 0
    arm_play = arm_to_play()
    best_res = best_response ()
    for i in range (0, num_arms):
        loss += W[i]*calcola_U(i)/2
    L[arm_play] += (4*(l**2)*(m**2) - loss)/1000
    return (4*(l**2)*(m**2) - loss)/1000

In [2]:
len([0,1])

2

In [3]:
s = theta([0.1, 0.4, 0.5])
p = theta([0.4, 0.6, 0.2])
q = theta([0.01, 0.1, 0.8])

In [4]:
a = arm([1,2,3])
b = arm([2,5,1])
c = arm([1, 3, 1])
w = [0.34, 0.33, 0.33]
V = design_matrix(w)

In [5]:
np.linalg.norm(c.arm_vector)

3.3166247903554

In [6]:
for i in range (0, num_arms):
    print(A[i].best_theta)

2
1
1


In [7]:
arm_to_play()

0

In [8]:
for i in range (0, num_arms):
    print(np.matmul(A[i].arm_vector, Th[theta_vero].vector))

2.4
2.7
1.8000000000000003


In [9]:
def h():
    return np.dot(W, L)

def mix_loss():
    sum = 0
    for i in range (0, num_arms):
        sum += W[i] * math.exp(-eta * L[i])
    return -1/eta * np.log(sum)

def mixability_gap():
    m_g = h() - mix_loss()
    Mixability_Gap.append(m_g)

def new_eta():
    if t>0:
        sum = 0
        for i in range (0, t):
            sum += Mixability_Gap[i]
        eta = math.log(dimension_arm)/sum
        
def generate_w():
    sum=0
    for i in range (0, num_arms):
        W[i] =math.exp(-eta * L[i])/num_arms
        sum += math.exp(-eta * L[i])/num_arms
    for i in range (0, num_arms):
        W[i] = W[i]/sum
    return W

In [10]:
a.pull_arm()

2.433430052087127

In [11]:
t += 1
stima_theta()
best_answer()
curr = arm_to_play()
mixability_gap()
generate_w()
err = calcola_loss()
print(err)
A[curr].pull_arm()

2.759705705014282


D:\Programmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


1.2400455380542077

In [12]:
t += 1
stima_theta()
best_answer()
curr = arm_to_play()
mixability_gap()
new_eta()
generate_w()
err = calcola_loss()
print(err)
A[curr].pull_arm()

2.484854809475676


1.2847344994322527

In [13]:
N

[1, 1, 1]

In [14]:
normalize(N)

array([0.33333333, 0.33333333, 0.33333333])

In [15]:
for i in range (3000):
    t += 1
    stima_theta()
    best_answer()
    curr = arm_to_play()
    mixability_gap()
    new_eta()
    generate_w()
    err = calcola_loss()
    print(err)
    A[curr].pull_arm()

2.5050628199701905
2.476871733944692
2.488887317447068
2.489218324960594
2.4599956921185555
2.4087160777967154
2.449005020299387
2.363806923940477
2.4715977675032406
2.5068923282477154
2.4255720022559686
2.447862767999012
2.4810868804833937
2.4057559604849525
2.4193390647474224
2.441694705613166
2.418963085369277
2.429687388687903
2.443687021017879
2.439568514483766
2.454446153942047
2.4482619665260006
2.4312208102438335
2.460744040455517
2.4585710410128394
2.436070933333779
2.410222690779843
2.4677288903871637
2.4403540999628963
2.389338194671778
2.4659561051588104
2.412081234343361
2.396139028376203
2.4443587964139413
2.463042894712024
2.3901845863864195
2.4163209455074477
2.415677762102786
2.414423007667327
2.3763632101279346
2.4400582576631993
2.4108231460284117
2.4067269528434982
2.410366840794597
2.440572607193048
2.3913325217321986
2.3855349166455535
2.4214678268351535
2.3929596240984736
2.4418180179984494
2.419754715977413
2.365506697975638
2.461499438544701
2.413373938498029
2

2.317311830759846
2.302513950105851
2.2665411595307488
2.2603973150398113
2.2982834704629194
2.234507861644818
2.359405156195826
2.354399756974925
2.297108847179731
2.3139817247666086
2.2717964066651595
2.2899096799017356
2.266920425841506
2.292292449765455
2.3100558856368734
2.3032414760214226
2.305928614847652
2.27414315404834
2.3139242803375724
2.3163917186984024
2.289228597018393
2.2871828086559924
2.2859352758388995
2.3034845301816587
2.3367713836341992
2.2630650857494494
2.2512280080571356
2.296421708859958
2.26483697934331
2.288593175239111
2.271006372683771
2.301996819561442
2.2400805235265584
2.2782135916994477
2.3057280341795057
2.285631905554673
2.3241045408598136
2.294866785409522
2.2539179389238204
2.3360025168284375
2.3273469745595428
2.234629282665668
2.3273892324104106
2.2853588989906632
2.2248311612495733
2.2901973927929165
2.3046157949809927
2.263709012579741
2.293911482220411
2.314200869337104
2.1862152247518125
2.31464514796998
2.3099804051945902
2.269225992683205
2

2.1525949390654144
2.194471980576258
2.125188660864111
2.175587589169193
2.1643156948311253
2.1141865897853322
2.1531134521980175
2.143081968479766
2.107278561434808
2.1746606799143886
2.153988814809187
2.1418624204443177
2.193347656131718
2.142155623790237
2.142877605795826
2.165321939323181
2.1743673381881607
2.1181449424459213
2.1449454295953494
2.141074596857121
2.126687104103583
2.196490678048951
2.1682175606073613
2.1104704023443355
2.1677276538134196
2.168495679819188
2.1114333660557296
2.1692236051340474
2.1554639794617025
2.105610564705041
2.159526172374102
2.183622433980046
2.122704891489958
2.1564115871247362
2.1475652371230542
2.0994582045893453
2.134709402142078
2.140009074210498
2.1172137747905033
2.1247009323359634
2.125706972231158
2.111279761228684
2.1363384302232613
2.1793743113640516
2.110706351318943
2.1524398692438087
2.1131317396418967
2.1089139235185295
2.156274631360031
2.1636896950335514
2.1130044825538863
2.158817751544918
2.1391631733959384
2.1250179383898393

2.0427322989567376
2.019738984560523
2.06557583692165
2.0533548423308448
2.0088012767409076
2.0440460397760707
2.0408248045357404
2.01697669735377
2.0603997944828394
2.0465798629015652
2.0124195372905667
2.0553283874870925
2.0460296215142697
2.01710187685076
2.066741884733221
2.0654343378701783
2.0364043895782835
2.0564922078004546
2.0374087038261646
2.0312509562230296
2.083892907499812
2.0329348185419978
2.03219375483834
2.07309284943317
2.069192977762691
2.0371284861771315
2.082486472501533
2.0609214782088685
2.0293249262674453
2.0817748288592584
2.051709866061667
2.0434136633301523
2.034976123406989
2.022200709781967
2.0214007281910815
2.054671142868446
2.0505638982555547
2.0037455744688066
2.0630077318433466
2.031773674657447
1.994175655120562
2.062308777011237
2.0264413948354925
2.0388943872544236
2.077858789695294
2.0404264196693336
2.0200242978472973
2.0582460922030696
2.0270138020394275
2.026642957189197
2.060423742968025
2.051686475219853
2.031287256443034
2.068156650712107
2.

1.968872949769404
2.006955497226311
1.9865549766527364
1.9702928593838462
1.9961103936602937
1.9912685810861208
1.9820339469146986
1.9791864762956026
1.98526349609942
1.9714444044847972
1.9982484459597223
1.997318127356154
1.9680533615800249
2.008922377340278
1.9780738974613368
1.9555556582041642
1.986310066687426
1.9939746820631599
1.972344619154806
2.010904640247557
1.9830284176057813
1.9698664262988128
1.9584001918102754
1.9885274517777174
1.9587657316164604
1.9900179842189336
1.982089076224658
1.980426894762791
2.00052168674562
1.9804708871813923
1.9622580414730704
1.9937755031981919
1.986400472336472
1.965423377752119
1.9933157656058584
1.9872636240935797
1.9809679581597044
2.0048806486625677
1.9763790417136256
1.9747200030374832
2.001858928480347
1.984194450494777
1.9867172878827417
1.9764709435048016
1.9868184622113705
1.9802516884055819
1.9903049377150506
1.9913909513063364
1.9636986717061455
1.9985230685864053
1.9835907584586476
1.9707085105330093
2.0069320744591055
1.98265193

1.9470195505682297
1.9528884263193325
1.9291471232236062
1.9770379530813857
1.944538322460904
1.9303199608964823
1.9579787891347298
1.9564495329093343
1.9328993183002336
1.9721765412258603
1.9515153017982838
1.9417895876735416
1.9633092172270392
1.9437605387376913
1.9350297522034572
1.9640954327864129
1.9366970450581238
1.939670183610998
1.9533864413778932
1.9418528253043574
1.936241444936654
1.9555506256353063
1.9401489054648962
1.922496251516686
1.958538289118347
1.948319153933223
1.947409700690505
1.9549812326643674
1.947748173342407
1.9258491705265495
1.9497757543703105
1.947312184744725
1.9416062871167488
1.9738004466311867
1.9405369129565928
1.9330228011060784
1.9613275315286802
1.9481746638116997
1.931212669802505
1.9417007947800597
1.9485473293617723
1.930720755659426
1.963596943086109
1.9421201494828746
1.9422017567624972
1.9558000873151753
1.9450696516955508
1.9354204672241413
1.970222306822684
1.9386088169137363
1.9293829878332143
1.9570702007569762
1.95126748517062
1.926582

1.9325656613920992
1.9171318160485726
1.909339198034457
1.9197844332813416
1.9151156376862442
1.909412723793149
1.9240011665998553
1.9111194283831954
1.9040910588836766
1.926333046943518
1.9140460674850828
1.9099687821094817
1.932377589114749
1.9213325887856776
1.9134863446947379
1.9283309240762552
1.9246937897416048
1.909674140451421
1.927493663291076
1.9253163725306721
1.9036791559019361
1.941924180766281
1.921209520045239
1.9040353685299
1.9275276381308353
1.9100280283439863
1.9154197990689787
1.9258044594237038
1.9169303368091841
1.9155777031693304
1.9284711112943462
1.919241205331626
1.9052279993028869
1.9343966061371045
1.9213401927872193
1.9086229417686449
1.929187286716312
1.9131190596895347
1.913053721608137
1.919640074209763
1.9113967312542086
1.9120484104981006
1.9257189609938898
1.9230352403621138
1.9121278848716468
1.932228226909709
1.9157249704233843
1.9021131657717745
1.9267026758818195
1.9110198102264193
1.9198360369786391
1.9298677427903486
1.9127090278474703
1.9066215

In [16]:
print(theta_stimato)
print(Th[theta_vero].vector)
print(best_answer())
print(best_arm())
print(W)

[0.34406907 0.30810474 0.49244165]
[0.1, 0.4, 0.5]
1
1
[0.050829593312210444, 0.8876562793003733, 0.06151412738741619]


In [17]:
b.pull_arm()
c.pull_arm()
b.pull_arm()

2.1921922556111073

In [18]:
R

[2.433430052087127,
 1.2400455380542077,
 1.2847344994322527,
 3.380474396531333,
 2.4087261255897543,
 3.7837688708772275,
 2.8427212717946055,
 0.07565624547299277,
 3.48396134219833,
 3.0962737107924005,
 1.5359590214798118,
 2.9152276182944736,
 2.0954147849611746,
 1.8151390198822912,
 1.7165310397221656,
 1.8384007605146726,
 3.134698462772378,
 3.112486846414956,
 3.2666825266604094,
 2.331067462327311,
 2.5064741513099844,
 1.9671750859894193,
 1.5304429336321037,
 0.6629931451602022,
 2.280729228366372,
 1.2627548423845245,
 2.1102996208810385,
 1.8232816230027273,
 2.492205209275001,
 2.532585212140055,
 2.716155862026431,
 0.541997868309966,
 3.7314054505820824,
 2.7834470726106324,
 1.8134540513285422,
 2.911321243411364,
 1.8950982502306735,
 3.3836250245333117,
 3.5547148422550503,
 1.265054219674185,
 2.804150814075272,
 1.4591454396837151,
 1.478300191689005,
 4.087502016418725,
 1.966205873587359,
 2.088780632548553,
 3.170714330178157,
 3.430449983365588,
 2.627532893

In [19]:
N

[1001, 1003, 1002]

In [20]:
W = [1.1, 2.6, 1.3]

In [21]:
arm_to_play()

0

In [22]:
t = 4
stima_theta()

array([ 0.00120047, -0.00062319,  0.00195133])

In [23]:
for i in range (0, num_arms):
    print(best_response(i))

TypeError: best_response() takes 0 positional arguments but 1 was given

In [ ]:
best_response(0)

In [ ]:
np.array(theta_stimato) - np.array(Th[best_response(0)].vector)

In [ ]:
np.array(A[0].arm_vector)

In [ ]:
p1 = np.matmul((np.array(theta_stimato) - np.array(Th[best_response(0)].vector)), np.array(A[0].arm_vector))
p1

In [ ]:
math.sqrt(2*beta(t, 1/t**3))

In [ ]:
np.sqrt(2*beta(t, 1/t**3))

In [ ]:
k = 1000
time = k
delta = 1/k**3


In [ ]:
n = np.abs(np.random.normal(0, 1))
n

In [ ]:
math.sqrt(math.log10(1/delta))

In [ ]:
dimension_arm/2*math.log10(1 + (time*(l**2))/(n*dimension_arm))

In [ ]:
math.sqrt(n/2)*m

In [ ]:
beta(k, 1/k**3)

In [ ]:
p2 = np.sqrt(2*beta(t, 1/t**3))*norm_2_mat(np.array(A[0].arm_vector), design_matrix(N) + np.random.normal(0, 1)*np.identity(d))

In [ ]:
calcola_U(0, 2)

In [ ]:
calcola_U(1, 2)

In [ ]:
calcola_U(2, 2)

In [ ]:
for i in range (0 , T//3 - t):
    a.pull_arm()
    b.pull_arm()
    c.pull_arm()
    t += 3

In [ ]:

stima_theta()

In [ ]:
for i in range (0, num_arms):
    print(best_response(i))

In [ ]:
for i in range (0, num_arms):
    print(np.matmul(A[i].arm_vector, Th[theta_vero].vector))

In [ ]:
for i in range (0, num_arms):
    print(calcola_U(i, best_response(i))/100000000)